In [3]:
import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
import operator

import pyarrow as pa
import pyarrow.parquet as pq

Traemos la data necesaria

In [4]:
df_reviews = pd.read_parquet("../Datasets/reviews_analisis_sentiment.parquet")
df_items = pd.read_parquet("../Datasets/australian_items_limpio.parquet")
df_games = pd.read_parquet("../Datasets/steam_games_limpio.parquet")

In [5]:
df_reviews.head(2)

,user_id,user_url,reviews_item_id,reviews_recommend,reviews_date,year,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,2011-11-05,2011,2
1,js41637,http://steamcommunity.com/id/js41637,251610,True,2014-06-24,2014,2


In [6]:
df_items.head(2)

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0


Para el modelo de recomendación se busca obtener una lista de 5 juegos a partir del id de un producto, o de un usuario  

Con la intencion de crear un puntaje que nos permita clasificar los juegos de una manera eficiente, vamos a crear una clasificacion tomando en cuenta el analisis de sentimiento y los reviews_recommend, para poder obtener esta escala:

1 si el análisis de sentimiento es negativo ya sea que este recomendado o no (True o False)

2 si el análisis de sentimiento es neutral y no es recomendado (False)

3 si el análisis de sentimiento es neutral pero es recomendado (True)

4 si el análisis de sentimiento es positivo y no es recomendado (False)

5 si el análisis de sentimiento es positivo y es recomendado (True)

Creamos la siguiente funcion

In [7]:
def calcula_rating(row):
   
    if row["sentiment_analysis"] == 0 and not row["reviews_recommend"]:
        return 1
    elif row["sentiment_analysis"] == 0 and row["reviews_recommend"]:
        return 1
    elif row["sentiment_analysis"] == 1 and not row["reviews_recommend"]:
        return 2
    elif row["sentiment_analysis"] == 1 and row["reviews_recommend"]:
        return 3
    elif row["sentiment_analysis"] == 2 and not row["reviews_recommend"]:
        return 4
    elif row["sentiment_analysis"] == 2 and row["reviews_recommend"]:
        return 5
    else:
        return None

Aplicamos la funcion a un anueva columna en el dataset

In [8]:
df_reviews["rating"] = df_reviews.apply(calcula_rating, axis=1)
df_reviews

,user_id,user_url,reviews_item_id,reviews_recommend,reviews_date,year,sentiment_analysis,rating
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,2011-11-05,2011,2,5
1,js41637,http://steamcommunity.com/id/js41637,251610,True,2014-06-24,2014,2,5
3,doctr,http://steamcommunity.com/id/doctr,250320,True,2013-10-14,2013,2,5
4,maplemage,http://steamcommunity.com/id/maplemage,211420,True,2014-04-15,2014,1,3
5,Wackky,http://steamcommunity.com/id/Wackky,249130,True,2014-05-05,2014,1,3
...,...,...,...,...,...,...,...,...
231273,kushikushigani,http://steamcommunity.com/id/kushikushigani,332310,True,2015-12-30,2015,1,3
231291,How51,http://steamcommunity.com/id/How51,440,True,2014-08-15,2014,0,1
231293,76561198111410893,http://steamcommunity.com/profiles/76561198111...,304930,True,2014-08-02,2014,2,5
231419,zaza147,http://steamcommunity.com/id/zaza147,265630,True,2015-07-31,2015,2,5


Seleccionamos oslo las columnas que necesitaremos

In [9]:
df_reviews = df_reviews[["user_id", "reviews_item_id", "rating"]]
df_reviews

,user_id,reviews_item_id,rating
0,76561197970982479,1250,5
1,js41637,251610,5
3,doctr,250320,5
4,maplemage,211420,3
5,Wackky,249130,3
...,...,...,...
231273,kushikushigani,332310,3
231291,How51,440,1
231293,76561198111410893,304930,5
231419,zaza147,265630,5


Seleccionamos los columnas necesarias de items

In [10]:
df_items = df_items[["item_id", "item_name"]]
df_items

,item_id,item_name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force
...,...,...
999995,333930,Dirty Bomb
999996,366844,Call of Duty: Black Ops III - Awakening DLC Pack
999997,377160,Fallout 4
999998,384190,ABZU


In [11]:
# se beorran los datos duplicados
df_items = df_items.drop_duplicates()

Unimops los datasets

In [12]:
df = df_reviews.merge(df_items, left_on="reviews_item_id", right_on="item_id", how="left")
df

,user_id,reviews_item_id,rating,item_id,item_name
0,76561197970982479,1250,5,1250,Killing Floor
1,js41637,251610,5,251610,Barbie™ Dreamhouse Party™
2,doctr,250320,5,250320,The Wolf Among Us
3,maplemage,211420,3,211420,Dark Souls: Prepare to Die Edition
4,Wackky,249130,3,249130,LEGO® MARVEL Super Heroes
...,...,...,...,...,...
47594,kushikushigani,332310,3,332310,LEGO® Worlds
47595,How51,440,1,NaN,NaN
47596,76561198111410893,304930,5,304930,Unturned
47597,zaza147,265630,5,265630,Fistful of Frags


In [13]:
df.isnull().sum()

user_id               0
reviews_item_id       0
rating                0
item_id            5619
item_name          5619
dtype: int64

Podemos observar que existe una cantidad de datos nulos en las columnas item, esto puede deberse a que algunos juego que recibieron recomendaciones, no existen en el dataframe

In [14]:
df = df.dropna(subset=['item_id'])

In [15]:
df.isnull().any()

user_id            False
reviews_item_id    False
rating             False
item_id            False
item_name          False
dtype: bool

Podemos verificar que eliminamos los registros nulos 

Seleccionamos solo las columnas que cumplen una funcion en nuestro analisis

In [16]:
df = df[["user_id", "item_name", "rating"]]
df

,user_id,item_name,rating
0,76561197970982479,Killing Floor,5
1,js41637,Barbie™ Dreamhouse Party™,5
2,doctr,The Wolf Among Us,5
3,maplemage,Dark Souls: Prepare to Die Edition,3
4,Wackky,LEGO® MARVEL Super Heroes,3
...,...,...,...
47593,76561198107177722,BattleBlock Theater,5
47594,kushikushigani,LEGO® Worlds,3
47596,76561198111410893,Unturned,5
47597,zaza147,Fistful of Frags,5


Procedemos a guardar la data en formato parquet

In [17]:
df.to_parquet("../Funciones/data/modelo_recomendacion.parquet")

Ya tenemos los datos seleccionados procedemos a crear la funcion y condiciones necesaria para cumplir con el modelo recomendado

In [18]:
# leemos la data
df_recomend = pd.read_parquet("../Funciones/data/modelo_recomendacion.parquet")
df_recomend

,user_id,item_name,rating
0,76561197970982479,Killing Floor,5
1,js41637,Barbie™ Dreamhouse Party™,5
2,doctr,The Wolf Among Us,5
3,maplemage,Dark Souls: Prepare to Die Edition,3
4,Wackky,LEGO® MARVEL Super Heroes,3
...,...,...,...
47593,76561198107177722,BattleBlock Theater,5
47594,kushikushigani,LEGO® Worlds,3
47596,76561198111410893,Unturned,5
47597,zaza147,Fistful of Frags,5


Colocamos los juegos como columnas, y sus valores serian el rating

In [19]:
piv = df.pivot_table(index=['user_id'], columns=['item_name'], values='rating')
piv

item_name,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,123 Slaughter Me Street,140,16 Bit Arena,200% Mixed Juice!,...,ibb & obb,inMomentum,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theHunter,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ionex--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2SV-vuLB-Kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Beave-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-GM-Dragon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zv_odd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zvanik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zwanzigdrei,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Vamos a restar la media de las calificaciones de un usuario y luego dividir por la diferencia entre el valor máximo y mínimo de las calificaciones

In [20]:
piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
piv_norm.fillna(0, inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:, (piv_norm != 0).any(axis=0)]
piv_norm

user_id,-Beave-,-GM-Dragon,-I_AM_EPIC-,00000000000000000001227,00690069006900,022899,03092002,04061993,0468313256,05041129,...,zimran,ziqan,zjaerya135,zombi_anon,zomgCoBfAce,zoom-the-flash,zsharoarkbr,zukuta,zvanik,zzoptimuszz
item_name,,,,,,,,,,,,,,,,,,,,,
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001 Spikes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Labours of Hercules,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
resident evil 4 / biohazard 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


A los datos de esta matriz normalizada se los convierte a un formato de matriz dispersa (sparse matrix) para reducir la memoria utilizada y mejorar la eficiencia en el manejo de grandes conjuntos de datos, especialmente cuando la mayoría de los valores en la matriz son ceros

In [21]:
piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
piv_sparse

<2617x6070 sparse matrix of type '<class 'numpy.float64'>'
	with 22583 stored elements in Compressed Sparse Row format>

Utilizaremos la similitud del coseno para evaluar los vectores 

In [22]:
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

Ingresamos las matrices en dataframes para poder lograr un mejor manejo

In [23]:
#item similarity dataframe
item_sim_df = pd.DataFrame(item_similarity, index = piv_norm.index, columns = piv_norm.index)
#user similarity dataframe
user_sim_df = pd.DataFrame(user_similarity, index = piv_norm.columns, columns = piv_norm.columns)

Funcion recomendacion_juego

In [24]:
def top_game(game):
        # Obtener la lista de juegos similares ordenados
    similar_games = item_sim_df.sort_values(by=game, ascending=False).iloc[1:6]

    count = 1
    contador = 1
    recomendaciones = {}
    
    for item in similar_games:
        if contador <= 5:
            item = str(item)
            recomendaciones[count] = item
            count += 1
            contador += 1 
        else:
            break
    return recomendaciones

Probamos la funcion

In [25]:
top_game('Killing Floor')

{1: '0RBITALIS',
 2: '10,000,000',
 3: '100% Orange Juice',
 4: '1001 Spikes',
 5: '12 Labours of Hercules'}

In [26]:
top_game('Barbie™ Dreamhouse Party™')

{1: '0RBITALIS',
 2: '10,000,000',
 3: '100% Orange Juice',
 4: '1001 Spikes',
 5: '12 Labours of Hercules'}

Funcion recomendacion_usuario

In [27]:
def similar_user_recs(user):

    # Verifica si el usuario está presente en las columnas de piv_norm (si no está, devuelve un mensaje)
    if user not in piv_norm.columns:
        return('No data available on user {}'.format(user))
    
    # Obtiene los usuarios más similares al usuario dado
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
    
    best = []  # Lista para almacenar los juegos mejor calificados por usuarios similares
    most_common = {}  # Diccionario para contar cuántas veces se recomienda cada juego
    
    # Para cada usuario similar, encuentra el juego mejor calificado y lo agrega a la lista 'best'
    for i in sim_users:
        max_score = piv_norm.loc[:, i].max()
        best.append(piv_norm[piv_norm.loc[:, i]==max_score].index.tolist())
    
    # Cuenta cuántas veces se recomienda cada juego
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1
    
    # Ordena los juegos por la frecuencia de recomendación en orden descendente
    sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)
    
    # Devuelve los 5 juegos más recomendados
    return sorted_list[:5]

Probando la funcion

In [28]:
similar_user_recs('zvanik')

[('Borderlands 2', 2),
 ('Call of Duty: World at War', 2),
 ('Counter-Strike: Global Offensive', 1),
 ('Crysis 2 Maximum Edition', 1),
 ('Goat Simulator', 1)]

In [29]:
similar_user_recs('76561197970982479')

[('Killing Floor', 8),
 ('Broforce', 1),
 ('Hotline Miami', 1),
 ('Metro 2033', 1)]

Procedemos a guadar las matrices para que puedan ser consumias por las funciones para la API

In [30]:
piv_norm.to_parquet("../Funciones/data/piv_norm.parquet")
user_sim_df.to_parquet("../Funciones/data/user_sim_df.parquet")
item_sim_df.to_parquet("../Funciones/data/item_sim_df.parquet")

In [40]:
df_games

,publisher,genres,title,reviews_url,price,early_access,id,developer,year
0,Kotoshiro,Action,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,4.99,False,761140,Kotoshiro,2018.0
0,Kotoshiro,Casual,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,4.99,False,761140,Kotoshiro,2018.0
0,Kotoshiro,Indie,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,4.99,False,761140,Kotoshiro,2018.0
0,Kotoshiro,Simulation,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,4.99,False,761140,Kotoshiro,2018.0
0,Kotoshiro,Strategy,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,4.99,False,761140,Kotoshiro,2018.0
...,...,...,...,...,...,...,...,...,...
32132,Laush Studio,Indie,Russian Roads,http://steamcommunity.com/app/610660/reviews/?...,1.99,False,610660,Laush Dmitriy Sergeevich,2018.0
32132,Laush Studio,Racing,Russian Roads,http://steamcommunity.com/app/610660/reviews/?...,1.99,False,610660,Laush Dmitriy Sergeevich,2018.0
32132,Laush Studio,Simulation,Russian Roads,http://steamcommunity.com/app/610660/reviews/?...,1.99,False,610660,Laush Dmitriy Sergeevich,2018.0
32133,SIXNAILS,Casual,EXIT 2 - Directions,http://steamcommunity.com/app/658870/reviews/?...,4.99,False,658870,"xropi,stev3ns",2017.0


In [32]:
mlb = MultiLabelBinarizer()
genre_features = pd.DataFrame(mlb.fit_transform(df_games['genres']), columns=mlb.classes_)

In [47]:
genre_features

,,&,;,A,C,D,E,F,G,I,...,n,o,p,r,s,t,u,v,w,y
0,0,0,0,1,0,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71546,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
71547,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
71548,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,1,0,0,0
71549,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


In [50]:
def recomendacion_juego(id_producto, num_recomendaciones=5):
    if id_producto not in genre_features.index:
        return {"error": "El juego no se encuentra en la base de datos."}
    # Resto del código para calcular la similitud del coseno y obtener las recomendaciones
    juego_seleccionado = np.array(genre_features.loc[id_producto].values).reshape(1, -1)
    similaridades = cosine_similarity(juego_seleccionado, genre_features)

    juegos_similares_indices = similaridades.argsort()[0][-num_recomendaciones:][::-1]
    juegos_recomendados = df_games.iloc[juegos_similares_indices, :]

    lista = [{"id": row['id'], "title": row['title']} for index, row in juegos_recomendados.iterrows()]

    return lista

In [52]:
recomendacion_juego(3000)

[{'id': '636132',
  'title': 'X-Plane 11 - Add-on: Aerosoft Airport Bonaire Flamingo'},
 {'id': '747910', 'title': 'Disassembly 3D'},
 {'id': '603550', 'title': 'Schlag den Star - Das Spiel'},
 {'id': '271412', 'title': 'Rocksmith® 2014 – Aerosmith - “Oh Yeah”'},
 {'id': '342832', 'title': 'Rocksmith® 2014 – All That Remains - “Two Weeks”'}]

In [53]:
genre_features.to_parquet("../Funciones/data/genre_features.parquet")